In [1]:
import pickle
import pandas as pd
import numpy as np
import datetime  
import pickle
from math import floor
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDRegressor, Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sqlalchemy import create_engine
from sqlalchemy import text
import csv

Inserting data base information and connecting to it

In [2]:
URI = "database-2.ckmnj1f5m6qh.eu-west-1.rds.amazonaws.com"
PORT = "3306"
DB = "backupdata"
USER = "admin"
PASSWORD = "DublinBikes3"

engine = create_engine("mysql://{}:{}@{}:{}/{}".format(USER, PASSWORD, URI, PORT, DB), echo=True)

In [3]:
# Get the data
availability_data = engine.execute("SELECT * FROM availability2;")

# Create CSV file
filename = "bike_data.csv"

# Open the CSV file in write mode
with open(filename, mode='w', newline='') as file:

    # Create a CSV writer object
    writer = csv.writer(file)

    # Write the header row (optional)
    writer.writerow(['number', 'available_bikes', 'available_bike_stands', 'status', 'datetime'])

    # Write the data rows
    for row in availability_data:
        writer.writerow(row)

2023-04-11 11:13:00,540 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2023-04-11 11:13:00,542 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-11 11:13:00,570 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2023-04-11 11:13:00,571 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-11 11:13:00,613 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2023-04-11 11:13:00,616 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-11 11:13:00,645 INFO sqlalchemy.engine.Engine SELECT * FROM availability2;
2023-04-11 11:13:00,647 INFO sqlalchemy.engine.Engine [raw sql] ()


In [4]:
dfa = pd.read_csv('bike_data.csv')
dfa.head()

,number,available_bikes,available_bike_stands,status,datetime
0,42,8,22,OPEN,1677850620
1,30,1,19,OPEN,1677850620
2,54,12,21,OPEN,1677850620
3,108,14,21,OPEN,1677850620
4,20,13,16,OPEN,1677850620


In [5]:
# Get the data
weather_data = engine.execute("SELECT * FROM weather2;")

# Create CSV file
filename = "weather_data.csv"

# Open the CSV file in write mode
with open(filename, mode='w', newline='') as file:

    # Create a CSV writer object
    writer = csv.writer(file)

    # Write the header row (optional)
    writer.writerow(['main', 'description', 'temp', 'visibility', 'wind_speed', 'wind_direction', 'clouds', 'datetime'])

    # Write the data rows
    for row in weather_data:
        writer.writerow(row)

2023-04-11 11:13:20,816 INFO sqlalchemy.engine.Engine SELECT * FROM weather2;
2023-04-11 11:13:20,817 INFO sqlalchemy.engine.Engine [raw sql] ()


In [6]:
dfw = pd.read_csv('weather_data.csv')
dfw.head()

,main,description,temp,visibility,wind_speed,wind_direction,clouds,datetime
0,Clouds,broken clouds,279.84,10000.0,4.12,70.0,75.0,1677850621
1,Clouds,broken clouds,279.84,10000.0,4.12,70.0,75.0,1677850922
2,Clouds,broken clouds,279.84,10000.0,4.12,70.0,75.0,1677851223
3,Clouds,broken clouds,279.84,10000.0,4.12,70.0,75.0,1677851524
4,Clouds,broken clouds,279.84,10000.0,4.12,70.0,75.0,1677851825


In [7]:
# Round these off so date time is to the nearest hour.
dfw['datetime'] = pd.to_datetime(dfw['datetime'], unit = 's')
dfw['datetime'] = dfw['datetime'].dt.floor('H')
dfw.head()

dfa['datetime'] = pd.to_datetime(dfw['datetime'], unit = 's')
dfa['datetime'] = dfa['datetime'].dt.floor('H')
dfa.head()

,number,available_bikes,available_bike_stands,status,datetime
0,42,8,22,OPEN,2023-03-03 13:00:00
1,30,1,19,OPEN,2023-03-03 13:00:00
2,54,12,21,OPEN,2023-03-03 13:00:00
3,108,14,21,OPEN,2023-03-03 13:00:00
4,20,13,16,OPEN,2023-03-03 13:00:00


In [8]:
# merge the two dataframes on the common column "key"
df = pd.merge(dfa, dfw, on='datetime')
df.head()

,number,available_bikes,available_bike_stands,status,datetime,main,description,temp,visibility,wind_speed,wind_direction,clouds
0,42,8,22,OPEN,2023-03-03 13:00:00,Clouds,broken clouds,279.84,10000.0,4.12,70.0,75.0
1,42,8,22,OPEN,2023-03-03 13:00:00,Clouds,broken clouds,279.84,10000.0,4.12,70.0,75.0
2,42,8,22,OPEN,2023-03-03 13:00:00,Clouds,broken clouds,279.84,10000.0,4.12,70.0,75.0
3,42,8,22,OPEN,2023-03-03 13:00:00,Clouds,broken clouds,279.84,10000.0,4.12,70.0,75.0
4,42,8,22,OPEN,2023-03-03 13:00:00,Clouds,broken clouds,279.84,10000.0,4.12,70.0,75.0


In [9]:
# Add a column for the day of the week (0=Monday, 6=Sunday)
df['dayofweek'] = df['datetime'].dt.dayofweek

# Change day of the week to weekday or weekend
df['weekday_or_weekend'] = df['dayofweek'].apply(lambda x: 'weekday' if x < 5 else 'weekend')
df = df.drop('dayofweek', axis=1)
# Add a column for the hour of the day (0-23)
df['hour'] = df['datetime'].dt.hour
df = df.drop('datetime', axis = 1)
df.head()

,number,available_bikes,available_bike_stands,status,main,description,temp,visibility,wind_speed,wind_direction,clouds,weekday_or_weekend,hour
0,42,8,22,OPEN,Clouds,broken clouds,279.84,10000.0,4.12,70.0,75.0,weekday,13
1,42,8,22,OPEN,Clouds,broken clouds,279.84,10000.0,4.12,70.0,75.0,weekday,13
2,42,8,22,OPEN,Clouds,broken clouds,279.84,10000.0,4.12,70.0,75.0,weekday,13
3,42,8,22,OPEN,Clouds,broken clouds,279.84,10000.0,4.12,70.0,75.0,weekday,13
4,42,8,22,OPEN,Clouds,broken clouds,279.84,10000.0,4.12,70.0,75.0,weekday,13


In [10]:
df = pd.get_dummies(df, columns=['status', 'main', 'description', 'weekday_or_weekend'])

In [11]:
# Split data into training and testing data
X = df.drop('available_bikes', axis = 1)
y = df['available_bikes']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [12]:
# create and fit the model
model = LogisticRegression()
model.fit(X_train, y_train)

# make predictions
y_pred = model.predict(X_test)

/Users/josephtummon/mambaforge/envs/comp30380/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [13]:
# Calculate root mean squared error (RMSE)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("Root Mean Squared Error (RMSE):", rmse)

# Calculate R-squared score
r2 = r2_score(y_test, y_pred)
print("R-squared score:", r2)

print(mean_absolute_error(y_test, y_pred))

Root Mean Squared Error (RMSE): 12.005592151332213
R-squared score: -1.1217706500616464
9.02789621147521


In [14]:
# create and fit the model
model = LinearRegression()
model.fit(X_train, y_train)

# make predictions
y_pred = model.predict(X_test)

# Calculate root mean squared error (RMSE)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("Root Mean Squared Error (RMSE):", rmse)

# Calculate R-squared score
r2 = r2_score(y_test, y_pred)
print("R-squared score:", r2)

Root Mean Squared Error (RMSE): 6.696662564896554
R-squared score: 0.33984168401213377


In [15]:
# create and fit the model
model = RandomForestRegressor()
model.fit(X_train, y_train)

# make predictions
y_pred = model.predict(X_test)

# Calculate root mean squared error (RMSE)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("Root Mean Squared Error (RMSE):", rmse)

# Calculate R-squared score
r2 = r2_score(y_test, y_pred)
print("R-squared score:", r2)

Root Mean Squared Error (RMSE): 0.06124196790769755
R-squared score: 0.9999447885006064


In [17]:
# X_train and y_train are the training data
model = SGDRegressor()
model.fit(X_train, y_train)
# Serialize model object into a file called model.pkl on disk using pickle
with open('model.pkl', 'wb') as handle:
    pickle.dump(model, handle, pickle.HIGHEST_PROTOCOL)
# pickle.HIGHEST_PROTOCOL using the highest available protocol
# (we used wb to open file as binary and use a higher pickling protocol)


In [28]:
feature_names = []
for i in df.columns:
    feature_names.append(i)

In [36]:
tester = []
for i in range(36):
    tester.append(i)
tester = [tester]

tester_named = np.rec.array(tester, dtype=[(name, float) for name in feature_names])

# Convert categorical variable to dummy variables
dummy_cols = pd.get_dummies(tester_named['categorical_var'])
tester_named = pd.concat([tester_named, dummy_cols], axis=1)
tester_named.drop('categorical_var', axis=1, inplace=True)

ValueError: no field of name categorical_var

In [37]:
model.predict(tester_named)

/Users/josephtummon/mambaforge/envs/comp30380/lib/python3.9/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but SGDRegressor was fitted with feature names
  warnings.warn(


ValueError: dtype='numeric' is not compatible with arrays of bytes/strings.Convert your data to numeric values explicitly instead.

In [18]:
# Gets features in arrays of [station number, days, hours]
def get_features():
    # Get datetime and station number for data
    features_raw = engine.execute("SELECT datetime, number FROM availability2;")

    # Create list to store station numbers, days and hours in.
    numbers_days_hours = []
    for row in features_raw:
        weather_raw = engine.execute("SELECT wind_speed FROM weather2 WHERE datetime BETWEEN %s AND %S", (row[0]-1, row[0]+2))
        # Will store each trio in
        number_day_hour_wind = []
        
        # Split epoch time into day and hour
        epoch_time = row[0]
        day = (floor(epoch_time / 86400) + 4) % 7
        number_day_hour.append(row[1])
        number_day_hour.append(day)
        number_day_hour.append(datetime.datetime.fromtimestamp(row[0]).hour)
        numbers_days_hours.append(number_day_hour)
    return numbers_days_hours

def get_availability():
    availability_raw = engine.execute("SELECT available_bikes FROM availability2;")
    # This returns the number of available bikes
    availability = []
    for row in availability_raw:
        availability.append(row[0])
    return availability
                                     